In [1]:
# 작성자 : 김형기

In [2]:
# -*- coding: utf-8 -*-
import os, sys, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from glob import glob

from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import figure, show, output_notebook, save
from bokeh.models import ColumnDataSource, HoverTool, BoxAnnotation
from bokeh.palettes import Category10
from bokeh.layouts import layout
from bokeh.io.export import export_png

sys.path.append(os.path.abspath("../../code"))

import functions as fc

# pandas 설정
pd.set_option('display.max_columns', 100)

# matplotlib 설정
plt.rcParams['font.family'] = 'NanumGothic'  # 나눔고딕 설정
plt.rcParams['axes.unicode_minus'] = False   # 마이너스 기호 깨짐 방지

In [3]:
data_path = os.path.abspath("../../data")
os.path.exists(data_path)

True

# Insight

- 창업에 적합한 식재료 후보는 “평균 가격이 낮고, 변동성도 낮은 품목”이며, 단위가 일관된 품목이 분석에 유리함

# 1. 데이터 전처리

## 1.1 소비자 성연령대별 농식품 소비
> - 데이터 특징: 월별/시군구별 성연령대별 농식품 소비 금액

* 분석방향
> 1. 2023년도 1월 데이터만 추출
> 2. 시도 / 시군구 기준으로 총소비량 집계 : 지역별 소비량을 파악하기 위함

In [4]:
file_path = rf"{data_path}/raw/소비자 성:연령대별 농식품 소비/TB_SEX_AGE_CSP_SPL-2023.csv"
encoding_detected = fc.detect_encoding(file_path)
df_customer = pd.read_csv(file_path, encoding=encoding_detected)
df_customer.columns = df_customer.columns.str.lower()
df_customer.fillna(value=0, inplace=True)
df_customer

,base_ym,csp_area_wsido_nm,csp_area_sggu_nm,mle_csp_amnt,wo_csp_amnt,mle_csp_ncas,wo_csp_ncas,mle_y20_belo_csp_amnt,wo_y20_belo_csp_amnt,mle_y20_csp_amnt,wo_y20_csp_amnt,mle_y30_csp_amnt,wo_y30_csp_amnt,mle_y40_csp_amnt,wo_y40_csp_amnt,mle_y50_csp_amnt,wo_y50_csp_amnt,mle_y60_anm_csp_amnt,wo_y60_anm_csp_amnt
0,202301,강원도,강릉시,54318512540,35192082213,1994476,1407160,0.0,0.0,3205668124,3120134991,11974233470,6859970424,18705227771,11499744819,14052978614,9152004887,6380404561,4560227092
1,202301,강원도,고성군,4289882028,2692053893,169262,100847,0.0,0.0,285857369,199879152,933113653,498107310,1408335699,741820434,1123925863,775666137,538649444,476580860
2,202301,강원도,동해시,17695890820,10884550622,657244,437090,0.0,0.0,834472421,585362873,3632323751,2127382541,6571857378,3678234398,4736473362,2862063903,1920763908,1631506907
3,202301,강원도,삼척시,12969673057,7227856764,433431,259636,0.0,0.0,482999492,391779503,2573880810,1521291929,4608484739,2356296062,3724915415,1970376703,1579392601,988112567
4,202301,강원도,속초시,31778675011,20895902232,988090,728452,0.0,0.0,1608628813,1787961216,7919481044,4705409815,11444004873,6905997519,7466321352,5104476826,3340238929,2392056856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2743,202312,충청북도,제천시,25769779721,16327747588,1044797,700958,7938916.0,5827208.0,1561803896,1248010603,5328292886,3431161080,8796420461,4729454779,7118600483,4389064980,2956723079,2524228938
2744,202312,충청북도,증평군,8874682521,4167143134,424651,200680,570015.0,810216.0,739485804,476676281,2633340460,936868410,2938365732,1332544908,1844563714,968377465,718356796,451865854
2745,202312,충청북도,진천군,18452192880,8434694870,912822,409812,2746450.0,1132205.0,1588100413,816921429,4654757470,2179749747,6232676395,2731704891,4382350148,1840749418,1591562004,864437180
2746,202312,충청북도,청주시,178834412404,109736268236,8391059,5673189,67929958.0,16823424.0,16982401804,14243586348,47237821046,26355532230,58701510780,34564804465,40839247840,24368853360,15005500976,10186668409


In [5]:
len(df_customer['csp_area_sggu_nm'].unique())

207

In [6]:
# 숫자형 데이터를 포함하는 컬럼들만 선택
numeric_columns = df_customer.select_dtypes(include=['int64', 'float64']).columns

# 선택된 컬럼들의 데이터 타입을 int64로 변경
df_customer[numeric_columns] = df_customer[numeric_columns].astype('int64')

In [7]:
df_customer_2301 = df_customer[(df_customer['base_ym'] == 202301)]
df_customer_2301

,base_ym,csp_area_wsido_nm,csp_area_sggu_nm,mle_csp_amnt,wo_csp_amnt,mle_csp_ncas,wo_csp_ncas,mle_y20_belo_csp_amnt,wo_y20_belo_csp_amnt,mle_y20_csp_amnt,wo_y20_csp_amnt,mle_y30_csp_amnt,wo_y30_csp_amnt,mle_y40_csp_amnt,wo_y40_csp_amnt,mle_y50_csp_amnt,wo_y50_csp_amnt,mle_y60_anm_csp_amnt,wo_y60_anm_csp_amnt
0,202301,강원도,강릉시,54318512540,35192082213,1994476,1407160,0,0,3205668124,3120134991,11974233470,6859970424,18705227771,11499744819,14052978614,9152004887,6380404561,4560227092
1,202301,강원도,고성군,4289882028,2692053893,169262,100847,0,0,285857369,199879152,933113653,498107310,1408335699,741820434,1123925863,775666137,538649444,476580860
2,202301,강원도,동해시,17695890820,10884550622,657244,437090,0,0,834472421,585362873,3632323751,2127382541,6571857378,3678234398,4736473362,2862063903,1920763908,1631506907
3,202301,강원도,삼척시,12969673057,7227856764,433431,259636,0,0,482999492,391779503,2573880810,1521291929,4608484739,2356296062,3724915415,1970376703,1579392601,988112567
4,202301,강원도,속초시,31778675011,20895902232,988090,728452,0,0,1608628813,1787961216,7919481044,4705409815,11444004873,6905997519,7466321352,5104476826,3340238929,2392056856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,202301,충청북도,제천시,21879959069,13240759477,914746,605159,0,0,988066194,885782448,4415050789,2859978621,7580448465,4400338254,6173505142,3343897803,2722888479,1750762351
225,202301,충청북도,증평군,7447068572,3864396127,334863,176960,0,0,476838407,417768337,2061067904,834652150,2525470152,1266890842,1727827501,933594443,655864608,411490355
226,202301,충청북도,진천군,15773784497,7344666147,733279,338722,0,0,997571219,627865760,3988859632,2030331591,5591992937,2130203004,3960126446,1813103025,1235234263,743162767
227,202301,충청북도,청주시,156284592510,96151903567,7345390,5045412,177096,284984,11533292177,10561027886,41317154428,23124670337,53075102722,31729223381,36607716070,21625849812,13751150017,9110847167


In [8]:
# 1. 시도, 시군구, 성별 소비 금액 추출
region_consumption = df_customer.loc[:, ['base_ym', 'csp_area_wsido_nm', 'csp_area_sggu_nm', 'mle_csp_amnt', 'wo_csp_amnt']]

# 2. 총 소비 금액 계산
# region_consumption['total_consumption'] = region_consumption['mle_csp_amnt'] + region_consumption['wo_csp_amnt']
region_consumption.loc[:, 'total_consumption'] = region_consumption['mle_csp_amnt'] + region_consumption['wo_csp_amnt']

region_consumption.reset_index(drop=True, inplace=True)

In [9]:
region_consumption

,base_ym,csp_area_wsido_nm,csp_area_sggu_nm,mle_csp_amnt,wo_csp_amnt,total_consumption
0,202301,강원도,강릉시,54318512540,35192082213,89510594753
1,202301,강원도,고성군,4289882028,2692053893,6981935921
2,202301,강원도,동해시,17695890820,10884550622,28580441442
3,202301,강원도,삼척시,12969673057,7227856764,20197529821
4,202301,강원도,속초시,31778675011,20895902232,52674577243
...,...,...,...,...,...,...
2743,202312,충청북도,제천시,25769779721,16327747588,42097527309
2744,202312,충청북도,증평군,8874682521,4167143134,13041825655
2745,202312,충청북도,진천군,18452192880,8434694870,26886887750
2746,202312,충청북도,청주시,178834412404,109736268236,288570680640


## 1.2 농수축산물 일자별 도소매 가격
> - 데이터 특징: 일별/품목/품종별 가격

* 특이사항
    - 시군구(25개) + 온라인(1개)

* EDA 방향
    - 도매, 온라인, 소매 별로 데이터를 나눠서 전처리 진행
    - 요식업 창업 리포트를 위해선 "도매"에서 식자재를 구매하는 것이 유리할 것 (가설) -> 온라인 / 소매 가격이랑 비교 예정

### 데이터 컬럼 설명

#### 기본 정보
| 컬럼명 | 설명 |
|--------|------|
| prce_reg_ymd | 가격등록일자 |
| mrkt_esnt_no | 시장고유번호 |
| mrkt_code | 시장코드 |
| mrkt_nm | 시장명 |

#### 지역 정보
| 컬럼명 | 설명 |
|--------|------|
| ctnp_code | 시도코드 |
| ctnp_nm | 시도명 |
| sggu_code | 시군구코드 |
| sggu_nm | 시군구명 |

#### 상품 정보
| 컬럼명 | 설명 |
|--------|------|
| pdlt_code | 품목코드 |
| pdlt_nm | 품목명 |
| spcs_code | 품종코드 |
| spcs_nm | 품종명 |
| bulk_grad_code | 산물등급코드 |
| bulk_grad_nm | 산물등급명 |
| pdlt_prce | 품목가격 |

#### 거래 구분
| 컬럼명 | 설명 |
|--------|------|
| wsrt_exmn_se_code | 도소매조사구분코드 |
| exmn_se_nm | 조사구분명 |

#### 단위 정보
| 컬럼명 | 설명 |
|--------|------|
| mtc_smt_unit_mg | 산지출하단위크기 |
| mtc_smt_unit_nm | 산지출하단위명 |
| whsl_smt_unit_mg | 도매출하단위크기 |
| whsl_smt_unit_nm | 도매출하단위명 |
| rtsl_smt_unit_mg | 소매출하단위크기 |
| rtsl_smt_unit_nm | 소매출하단위명 |
| evfd_fmpd_smt_unit_mg | 친환경농산물출하단위크기 |
| evfd_fmpd_smt_unit_nm | 친환경농산물출하단위명 |

#### 기타 정보
| 컬럼명 | 설명 |
|--------|------|
| dcnt_prce_yn | 할인가격여부 |
| etl_ldg_dt | ETL적재일시 |

In [10]:
# 농수축산물 일자별 도소매 가격이지 판매량을 나타내는 것은 아니다.
if not os.path.exists(f"{data_path}/preprocessed/농수축산물_일자별_도소매_가격_2023.parquet"):
    df_product = pd.DataFrame()
    for file_path in sorted(glob(f"{data_path}/raw/농수축산물 일자별 도소매 가격/2023*.csv"), reverse=True):
        encoding_detected = fc.detect_encoding(file_path)
        temp = pd.read_csv(file_path, encoding=encoding_detected)
        temp.columns = temp.columns.str.lower()
        df_product = pd.concat([df_product, temp])
    df_product.sort_values(by='prce_reg_ymd', ascending=True, inplace=True)
    df_product.reset_index(drop=True, inplace=True)
    df_product.to_parquet(f"{data_path}/preprocessed/농수축산물_일자별_도소매_가격_2023.parquet", index=False)
else:
    df_product = pd.read_parquet(f"{data_path}/preprocessed/농수축산물_일자별_도소매_가격_2023.parquet")

In [11]:
df_product

,prce_reg_ymd,mrkt_esnt_no,mrkt_code,mrkt_nm,ctnp_code,ctnp_nm,sggu_code,sggu_nm,pdlt_code,pdlt_nm,spcs_code,spcs_nm,wsrt_exmn_se_code,exmn_se_nm,bulk_grad_code,bulk_grad_nm,pdlt_prce,mtc_smt_unit_mg,mtc_smt_unit_nm,whsl_smt_unit_mg,whsl_smt_unit_nm,rtsl_smt_unit_mg,rtsl_smt_unit_nm,evfd_fmpd_smt_unit_mg,evfd_fmpd_smt_unit_nm,dcnt_prce_yn,etl_ldg_dt
0,20230102,98,330401,B-유통,33,충북,3311,청주,818,콩나물,1,풀무원 국산콩 콩나물 340g,1,소매,4,상품,2600,None,None,NaN,None,1.0,봉,NaN,None,N,20240102062408
1,20230102,607,220024,칠성,22,대구,2200,대구,241,건고추,0,화건,1,소매,4,상품,14600,600,g,30.0,kg,600.0,g,NaN,None,N,20240102062408
2,20230102,607,220024,칠성,22,대구,2200,대구,233,열무,0,열무,1,소매,4,상품,2000,None,None,4.0,kg,1.0,kg,NaN,None,N,20240102062408
3,20230102,607,220024,칠성,22,대구,2200,대구,232,당근,1,무세척,1,소매,5,중품,3000,None,None,20.0,kg,1.0,kg,1.0,kg,N,20240102062408
4,20230102,607,220024,칠성,22,대구,2200,대구,232,당근,1,무세척,1,소매,4,상품,4000,None,None,20.0,kg,1.0,kg,1.0,kg,N,20240102062408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1263938,20231229,63,220403,A-유통,22,대구,2200,대구,226,딸기,0,딸기,1,소매,4,상품,3390,None,None,2.0,kg,100.0,g,100.0,g,N,20240113065914
1263939,20231229,63,220403,A-유통,22,대구,2200,대구,225,토마토,0,토마토,1,소매,4,상품,9460,None,None,5.0,kg,1.0,kg,1.0,kg,N,20240113065914
1263940,20231229,63,220403,A-유통,22,대구,2200,대구,224,호박,1,애호박,1,소매,4,상품,2290,None,None,20.0,개,1.0,개,1.0,개,N,20240113065914
1263941,20231229,63,220403,A-유통,22,대구,2200,대구,223,오이,1,가시계통,1,소매,4,상품,21630,None,None,10.0,kg,10.0,개,10.0,개,N,20240113065914


In [12]:
# sgg_nm 개수
len(df_product['sggu_nm'].unique()), df_product['sggu_nm'].unique()

(26,
 array(['청주', '대구', '부산', '대전', '인천', '서울', '광주', '수원', '의정부', '춘천', '울산',
        '제주', '포항', '순천', '전주', '안동', '창원', '강릉', '세종', '온라인', '고양', '용인',
        '성남', '김해시', '천안', '김해'], dtype=object))

### 1.2.1 도매 데이터

In [13]:
df_dm = df_product[df_product['exmn_se_nm'] == '도매'].reset_index(drop=True)
df_dm

,prce_reg_ymd,mrkt_esnt_no,mrkt_code,mrkt_nm,ctnp_code,ctnp_nm,sggu_code,sggu_nm,pdlt_code,pdlt_nm,spcs_code,spcs_nm,wsrt_exmn_se_code,exmn_se_nm,bulk_grad_code,bulk_grad_nm,pdlt_prce,mtc_smt_unit_mg,mtc_smt_unit_nm,whsl_smt_unit_mg,whsl_smt_unit_nm,rtsl_smt_unit_mg,rtsl_smt_unit_nm,evfd_fmpd_smt_unit_mg,evfd_fmpd_smt_unit_nm,dcnt_prce_yn,etl_ldg_dt
0,20230102,6,210022,부전,21,부산,2100,부산,241,건고추,2,양건,2,도매,5,중품,750000,600,g,30.0,kg,600.0,g,NaN,None,N,20240102062408
1,20230102,6,210022,부전,21,부산,2100,부산,241,건고추,2,양건,2,도매,4,상품,775000,600,g,30.0,kg,600.0,g,NaN,None,N,20240102062408
2,20230102,6,210022,부전,21,부산,2100,부산,241,건고추,0,화건,2,도매,5,중품,575000,600,g,30.0,kg,600.0,g,NaN,None,N,20240102062408
3,20230102,6,210022,부전,21,부산,2100,부산,241,건고추,0,화건,2,도매,4,상품,650000,600,g,30.0,kg,600.0,g,NaN,None,N,20240102062408
4,20230102,6,210022,부전,21,부산,2100,부산,141,콩,1,흰 콩(국산),2,도매,4,상품,222000,40,kg,40.0,kg,500.0,g,NaN,None,N,20240102062408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168484,20231229,11,220021,북부도매,22,대구,2200,대구,315,느타리버섯,1,애느타리버섯,2,도매,4,상품,9000,None,None,2.0,kg,100.0,g,NaN,None,N,20240113065914
168485,20231229,11,220021,북부도매,22,대구,2200,대구,258,깐마늘(국산),3,깐마늘(대서),2,도매,5,중품,125000,None,None,20.0,kg,NaN,None,NaN,None,N,20240113065914
168486,20231229,11,220021,북부도매,22,대구,2200,대구,258,깐마늘(국산),3,깐마늘(대서),2,도매,4,상품,135000,None,None,20.0,kg,NaN,None,NaN,None,N,20240113065914
168487,20231229,13,220024,칠성,22,대구,2200,대구,241,건고추,0,화건,2,도매,5,중품,575000,600,g,30.0,kg,600.0,g,NaN,None,N,20240113065914


In [14]:
# 품목 > 품종
unique_species = df_dm['pdlt_nm'].unique()
print(f"\n총 품목 수: {len(unique_species)}")
print("\n품목 목록:")
print(unique_species)

unique_species = df_dm['spcs_nm'].unique()
print(f"\n총 품종 수: {len(unique_species)}")
print("\n품종 목록:")
print(unique_species)


총 품목 수: 70

품목 목록:
['건고추' '콩' '참깨' '찹쌀' '쌀' '녹두' '피마늘' '땅콩' '팥' '들깨' '고등어' '갈치' '명태' '물오징어'
 '무' '망고' '레몬' '당근' '딸기' '토마토' '호박' '열무' '붉은고추' '풋고추' '오이' '수박' '바나나' '단감'
 '고구마' '배' '사과' '새송이버섯' '팽이버섯' '참다래' '감자' '얼갈이배추' '상추' '시금치' '양배추' '배추'
 '파인애플' '느타리버섯' '멜론' '포도' '방울토마토' '감귤' '깐마늘(국산)' '양파' '파' '파프리카' '피망' '깻잎'
 '미나리' '생강' '새우' '전복' '굴' '건미역' '김' '건오징어' '북어' '건멸치' '메밀' '오렌지' '체리' '참외'
 '브로콜리' '알배기배추' '복숭아' '삼치']

총 품종 수: 103

품종 목록:
['양건' '화건' '흰 콩(국산)' '인도' '일반계' '흰 콩(수입)' '수입' '국산' '난지(남도)' '붉은 팥(수입)'
 '난지(대서)' '중국' '백색(국산)' '붉은 팥(국산)' '냉동' '생선' '월동' '무세척' '딸기' '토마토' '쥬키니'
 '애호박' '세척(수입)' '열무' '붉은고추' '청양고추' '꽈리고추' '풋고추' '가시계통' '수박' '단감' '밤' '신고'
 '오이맛고추' '후지' '새송이버섯' '팽이버섯' '그린 뉴질랜드' '수미' '얼갈이배추' '청' '적' '시금치' '양배추'
 '느타리버섯' '멜론' '샤인머스켓' '대추방울토마토' '노지' '애느타리버섯' '깐마늘(대서)' '양파' '대파' '파프리카'
 '깻잎' '미나리' '쪽파' '흰다리(수입)' '전복' '굴' '건미역' '마른김' '건오징어' '황태' '건멸치' '냉동(수입)'
 '취청' '다다기계통' '메밀(수입)' '네이블 미국' '깐마늘(남도)' '레드글로브 페루' '대지마' '시설' '참외' '햇양파'
 '레드글로브 칠레' '봄' '수미(노지)' '양건(~23.5)' '20kg' '레드

In [15]:
df_dm[df_dm['pdlt_nm'].str.contains('고추')]['pdlt_nm'].unique()

array(['건고추', '붉은고추', '풋고추'], dtype=object)

In [16]:
df_dm['prce_reg_ymd'] = pd.to_datetime(df_dm['prce_reg_ymd'], format='%Y%m%d')
df_dm['base_ym'] = pd.to_datetime(df_dm['prce_reg_ymd']).dt.strftime('%Y%m')

In [17]:
df_dm.columns

Index(['prce_reg_ymd', 'mrkt_esnt_no', 'mrkt_code', 'mrkt_nm', 'ctnp_code',
       'ctnp_nm', 'sggu_code', 'sggu_nm', 'pdlt_code', 'pdlt_nm', 'spcs_code',
       'spcs_nm', 'wsrt_exmn_se_code', 'exmn_se_nm', 'bulk_grad_code',
       'bulk_grad_nm', 'pdlt_prce', 'mtc_smt_unit_mg', 'mtc_smt_unit_nm',
       'whsl_smt_unit_mg', 'whsl_smt_unit_nm', 'rtsl_smt_unit_mg',
       'rtsl_smt_unit_nm', 'evfd_fmpd_smt_unit_mg', 'evfd_fmpd_smt_unit_nm',
       'dcnt_prce_yn', 'etl_ldg_dt', 'base_ym'],
      dtype='object')

In [18]:
df_dm = df_dm[['base_ym', 'prce_reg_ymd', 'mrkt_esnt_no', 'mrkt_code', 'mrkt_nm', 'ctnp_code',
               'ctnp_nm', 'sggu_code', 'sggu_nm', 'pdlt_code', 'pdlt_nm', 'spcs_code',
               'spcs_nm', 'wsrt_exmn_se_code', 'exmn_se_nm', 'bulk_grad_code',
               'bulk_grad_nm', 'pdlt_prce', 'whsl_smt_unit_mg', 'whsl_smt_unit_nm',
               'dcnt_prce_yn']].sort_values(by='prce_reg_ymd', ascending=True).reset_index(drop=True)

In [19]:
df_dm

,base_ym,prce_reg_ymd,mrkt_esnt_no,mrkt_code,mrkt_nm,ctnp_code,ctnp_nm,sggu_code,sggu_nm,pdlt_code,pdlt_nm,spcs_code,spcs_nm,wsrt_exmn_se_code,exmn_se_nm,bulk_grad_code,bulk_grad_nm,pdlt_prce,whsl_smt_unit_mg,whsl_smt_unit_nm,dcnt_prce_yn
0,202301,2023-01-02,6,210022,부전,21,부산,2100,부산,241,건고추,2,양건,2,도매,5,중품,750000,30.0,kg,N
1,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,420,파인애플,2,수입,2,도매,5,중품,26000,12.0,kg,N
2,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,420,파인애플,2,수입,2,도매,4,상품,29000,12.0,kg,N
3,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,152,감자,1,수미,2,도매,4,상품,45000,20.0,kg,N
4,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,418,바나나,2,수입,2,도매,5,중품,24300,13.0,kg,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168484,202312,2023-12-29,9,210042,엄궁도매,21,부산,2100,부산,246,파,2,쪽파,2,도매,5,중품,8000,1.0,kg,N
168485,202312,2023-12-29,9,210042,엄궁도매,21,부산,2100,부산,246,파,2,쪽파,2,도매,4,상품,10000,1.0,kg,N
168486,202312,2023-12-29,9,210042,엄궁도매,21,부산,2100,부산,246,파,0,대파,2,도매,5,중품,4300,1.0,kg,N
168487,202312,2023-12-29,7,210031,공동어,21,부산,2100,부산,653,전복,0,전복,2,도매,5,중품,30600,1.0,kg,N


In [20]:
df_dm['whsl_smt_unit_nm'].unique()

array(['kg', '개', '속', '마리', 'kg(그물망 3포기)'], dtype=object)

In [21]:
df_dm.groupby('whsl_smt_unit_nm').size()

whsl_smt_unit_nm
kg             155614
kg(그물망 3포기)      1002
개                8258
마리               2410
속                1205
dtype: int64

In [22]:
df_dm['whsl_smt_unit_nm'] = df_dm['whsl_smt_unit_nm'].str.strip()

In [23]:
df_dm[df_dm['whsl_smt_unit_nm'] == 'kg(그물망 3포기)']['pdlt_nm'].unique()

array(['배추'], dtype=object)

In [24]:
df_dm[(df_dm['pdlt_nm'] == '배추') & (df_dm['mrkt_nm'].str.contains('가락')) & (df_dm['bulk_grad_code'] == 4) & (df_dm['base_ym'] == '202307')]

,base_ym,prce_reg_ymd,mrkt_esnt_no,mrkt_code,mrkt_nm,ctnp_code,ctnp_nm,sggu_code,sggu_nm,pdlt_code,pdlt_nm,spcs_code,spcs_nm,wsrt_exmn_se_code,exmn_se_nm,bulk_grad_code,bulk_grad_nm,pdlt_prce,whsl_smt_unit_mg,whsl_smt_unit_nm,dcnt_prce_yn
82970,202307,2023-07-03,2,110211,가락도매,11,서울,1101,서울,211,배추,1,봄,2,도매,4,상품,9800,10.0,kg,N
83648,202307,2023-07-04,2,110211,가락도매,11,서울,1101,서울,211,배추,1,봄,2,도매,4,상품,9400,10.0,kg,N
84362,202307,2023-07-05,2,110211,가락도매,11,서울,1101,서울,211,배추,1,봄,2,도매,4,상품,9200,10.0,kg,N
85145,202307,2023-07-06,2,110211,가락도매,11,서울,1101,서울,211,배추,1,봄,2,도매,4,상품,9000,10.0,kg,N
85525,202307,2023-07-07,2,110211,가락도매,11,서울,1101,서울,211,배추,1,봄,2,도매,4,상품,7800,10.0,kg,N
86413,202307,2023-07-10,2,110211,가락도매,11,서울,1101,서울,211,배추,1,봄,2,도매,4,상품,10800,10.0,kg,N
87112,202307,2023-07-11,2,110211,가락도매,11,서울,1101,서울,211,배추,1,봄,2,도매,4,상품,10600,10.0,kg,N
87550,202307,2023-07-12,2,110211,가락도매,11,서울,1101,서울,211,배추,1,봄,2,도매,4,상품,9800,10.0,kg,N
88502,202307,2023-07-13,2,110211,가락도매,11,서울,1101,서울,211,배추,1,봄,2,도매,4,상품,10400,10.0,kg,N
89278,202307,2023-07-14,2,110211,가락도매,11,서울,1101,서울,211,배추,1,봄,2,도매,4,상품,11400,10.0,kg,N


In [25]:
df_dm[(df_dm['pdlt_nm'] == '배추') & (df_dm['mrkt_nm'].str.contains('가락')) & (df_dm['bulk_grad_code'] == 4) & (df_dm['base_ym'] == '202308')]

,base_ym,prce_reg_ymd,mrkt_esnt_no,mrkt_code,mrkt_nm,ctnp_code,ctnp_nm,sggu_code,sggu_nm,pdlt_code,pdlt_nm,spcs_code,spcs_nm,wsrt_exmn_se_code,exmn_se_nm,bulk_grad_code,bulk_grad_nm,pdlt_prce,whsl_smt_unit_mg,whsl_smt_unit_nm,dcnt_prce_yn
97163,202308,2023-08-01,2,110211,가락도매,11,서울,1101,서울,211,배추,2,여름(고랭지),2,도매,4,상품,19600,10.0,kg,N
98045,202308,2023-08-02,2,110211,가락도매,11,서울,1101,서울,211,배추,2,여름(고랭지),2,도매,4,상품,21400,10.0,kg,N
98745,202308,2023-08-03,2,110211,가락도매,11,서울,1101,서울,211,배추,2,여름(고랭지),2,도매,4,상품,21600,10.0,kg,N
99474,202308,2023-08-04,2,110211,가락도매,11,서울,1101,서울,211,배추,2,여름(고랭지),2,도매,4,상품,23200,10.0,kg,N
100207,202308,2023-08-07,2,110211,가락도매,11,서울,1101,서울,211,배추,2,여름(고랭지),2,도매,4,상품,23600,10.0,kg(그물망 3포기),N
100784,202308,2023-08-08,2,110211,가락도매,11,서울,1101,서울,211,배추,2,여름(고랭지),2,도매,4,상품,23400,10.0,kg(그물망 3포기),N
101486,202308,2023-08-09,2,110211,가락도매,11,서울,1101,서울,211,배추,2,여름(고랭지),2,도매,4,상품,27200,10.0,kg(그물망 3포기),N
102320,202308,2023-08-10,2,110211,가락도매,11,서울,1101,서울,211,배추,2,여름(고랭지),2,도매,4,상품,32800,10.0,kg(그물망 3포기),N
102929,202308,2023-08-11,2,110211,가락도매,11,서울,1101,서울,211,배추,2,여름(고랭지),2,도매,4,상품,35600,10.0,kg(그물망 3포기),N
103643,202308,2023-08-14,2,110211,가락도매,11,서울,1101,서울,211,배추,2,여름(고랭지),2,도매,4,상품,19000,10.0,kg(그물망 3포기),N


In [26]:
df_dm['whsl_smt_unit_nm'] = df_dm['whsl_smt_unit_nm'].str.replace(r'(그물망 3포기)', '') # 그물망 3포기는 202308월부터 출하 단위로 변경되어, 일괄적으로 kg으로 변경
df_dm['whsl_smt_unit_mg'] = df_dm['whsl_smt_unit_mg'].astype(int)

In [27]:
# 도매출하단위 확인
df_dm['whsl_smt_unit_nm'].unique()

array(['kg', '개', '속', '마리'], dtype=object)

In [28]:
for unit_nm in df_dm['whsl_smt_unit_nm'].unique():
    items = df_dm[df_dm['whsl_smt_unit_nm'] == unit_nm]['pdlt_nm'].unique()
    print(f"\n출하단위: {unit_nm}")
    print(f"품목 수: {len(items)}개")
    print(f"품목 목록: {', '.join(items)}")


출하단위: kg
품목 수: 66개
품목 목록: 건고추, 파인애플, 감자, 바나나, 배추, 고구마, 배, 풋고추, 사과, 단감, 양배추, 미나리, 깻잎, 오이, 쌀, 고등어, 망고, 들깨, 참깨, 팥, 콩, 붉은고추, 새우, 전복, 토마토, 얼갈이배추, 파, 생강, 팽이버섯, 느타리버섯, 피마늘, 멜론, 파프리카, 피망, 방울토마토, 포도, 상추, 시금치, 새송이버섯, 양파, 딸기, 레몬, 깐마늘(국산), 감귤, 찹쌀, 건미역, 오렌지, 참다래, 건멸치, 열무, 당근, 무, 호박, 물오징어, 명태, 갈치, 굴, 땅콩, 메밀, 녹두, 체리, 참외, 브로콜리, 알배기배추, 복숭아, 삼치

출하단위: 개
품목 수: 3개
품목 목록: 수박, 오이, 호박

출하단위: 속
품목 수: 1개
품목 목록: 김

출하단위: 마리
품목 수: 2개
품목 목록: 건오징어, 북어


In [29]:
df_dm[df_dm['whsl_smt_unit_nm'] == '개']['pdlt_nm'].unique().tolist()

['수박', '오이', '호박']

In [30]:
df_dm[df_dm['whsl_smt_unit_nm'] == 'kg']['pdlt_nm'].unique()

array(['건고추', '파인애플', '감자', '바나나', '배추', '고구마', '배', '풋고추', '사과', '단감',
       '양배추', '미나리', '깻잎', '오이', '쌀', '고등어', '망고', '들깨', '참깨', '팥', '콩',
       '붉은고추', '새우', '전복', '토마토', '얼갈이배추', '파', '생강', '팽이버섯', '느타리버섯',
       '피마늘', '멜론', '파프리카', '피망', '방울토마토', '포도', '상추', '시금치', '새송이버섯',
       '양파', '딸기', '레몬', '깐마늘(국산)', '감귤', '찹쌀', '건미역', '오렌지', '참다래',
       '건멸치', '열무', '당근', '무', '호박', '물오징어', '명태', '갈치', '굴', '땅콩', '메밀',
       '녹두', '체리', '참외', '브로콜리', '알배기배추', '복숭아', '삼치'], dtype=object)

In [31]:
df_dm[df_dm['whsl_smt_unit_nm'] == '마리']['pdlt_nm'].unique()

array(['건오징어', '북어'], dtype=object)

In [32]:
df_dm[df_dm['whsl_smt_unit_nm'] == '속']['pdlt_nm'].unique()

array(['김'], dtype=object)

#### 1) EDA (5개 품목 선정)

In [33]:
# 5개 품목을 선정하여 도매 가격 추이 분석
selected_items = ['감자', '배추', '당근', '양파', '풋고추']

df = df_dm[df_dm['pdlt_nm'].isin(selected_items)].reset_index(drop=True) # 단위가격 산출
df.loc[:, 'unit_price'] = (df['pdlt_prce'] / df['whsl_smt_unit_mg'])
df

,base_ym,prce_reg_ymd,mrkt_esnt_no,mrkt_code,mrkt_nm,ctnp_code,ctnp_nm,sggu_code,sggu_nm,pdlt_code,pdlt_nm,spcs_code,spcs_nm,wsrt_exmn_se_code,exmn_se_nm,bulk_grad_code,bulk_grad_nm,pdlt_prce,whsl_smt_unit_mg,whsl_smt_unit_nm,dcnt_prce_yn,unit_price
0,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,152,감자,1,수미,2,도매,4,상품,45000,20,kg,N,2250.000000
1,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,211,배추,6,월동,2,도매,5,중품,4960,10,kg,N,496.000000
2,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,242,풋고추,4,오이맛고추,2,도매,5,중품,59000,10,kg,N,5900.000000
3,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,242,풋고추,4,오이맛고추,2,도매,4,상품,64000,10,kg,N,6400.000000
4,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,152,감자,1,수미,2,도매,5,중품,41000,20,kg,N,2050.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20718,202312,2023-12-29,9,210042,엄궁도매,21,부산,2100,부산,152,감자,1,수미(노지),2,도매,5,중품,39000,20,kg,N,1950.000000
20719,202312,2023-12-29,9,210042,엄궁도매,21,부산,2100,부산,211,배추,6,월동,2,도매,4,상품,8000,10,kg,N,800.000000
20720,202312,2023-12-29,9,210042,엄궁도매,21,부산,2100,부산,211,배추,6,월동,2,도매,5,중품,6500,10,kg,N,650.000000
20721,202312,2023-12-29,9,210042,엄궁도매,21,부산,2100,부산,245,양파,0,양파,2,도매,4,상품,19500,15,kg,N,1300.000000


In [34]:
df['bulk_grad_nm'].unique()

array(['상품', '중품'], dtype=object)

In [35]:
df['whsl_smt_unit_nm'].unique()

array(['kg'], dtype=object)

##### 1) 일자별 평균 도매 단가 계산

In [36]:
price_trend = df.groupby(['prce_reg_ymd', 'pdlt_nm'])['unit_price'].mean().reset_index()

In [37]:
price_trend

,prce_reg_ymd,pdlt_nm,unit_price
0,2023-01-02,감자,2258.000000
1,2023-01-02,당근,2059.333333
2,2023-01-02,배추,593.500000
3,2023-01-02,양파,1452.666667
4,2023-01-02,풋고추,6972.000000
...,...,...,...
1200,2023-12-29,감자,2168.500000
1201,2023-12-29,당근,1264.333333
1202,2023-12-29,배추,761.600000
1203,2023-12-29,양파,1291.333333


In [38]:
fc.plot_daily_price_with_moving_avg(price_trend, window=30, title="📈 일평균 주요 식재료 도매 단가 추이 (이동평균선 포함)", type="show")

Loading BokehJS ...

- 감자는 4월 ~ 6월 사이의 가격이 높아짐
- 풋고추는 급등락이 심함
- 배추의 경우 1월 ~ 8월까지는 안정적이다가 8월 ~ 10월에 급등하는 경향이 있음 -> 김장철의 영향
- 양파는 그 중 가장 안정품목
- 당근의 경우 8월 ~ 12월 사이에 가격이 급등하는 추세를 보임

##### 2) 변동계수 계산 : 단위 상관없이 가격의 상대적인 변동성 비교 (0.3 이상이면 변동성이 큼 / 0.05면 안정)

In [39]:
cv_summary_org, top_items_org, filtered_org = fc.calculate_cv_summary(price_trend)

##### 3. 시각화

In [40]:
cv_summary_org.head()

,base_ym,pdlt_nm,avg_unit_price,std_unit_price,cv,high_cv,avg_unit_price_shift,mom_increase,risk_flag_cv,risk_flag_mom
0,202301,감자,2213.411765,53.623757,0.024227,False,NaN,NaN,False,False
5,202302,감자,2251.802679,73.494318,0.032638,False,2213.411765,0.017345,False,False
10,202303,감자,2296.409091,35.333323,0.015386,False,2251.802679,0.019809,False,False
15,202304,감자,2908.300000,168.443056,0.057918,False,2296.409091,0.266456,False,True
20,202305,감자,3233.650000,433.715469,0.134126,True,2908.300000,0.111869,True,False


In [41]:
filtered_org.head()

,base_ym,pdlt_nm,avg_unit_price,std_unit_price,cv,high_cv,avg_unit_price_shift,mom_increase,risk_flag_cv,risk_flag_mom
0,202301,감자,2213.411765,53.623757,0.024227,False,NaN,NaN,False,False
5,202302,감자,2251.802679,73.494318,0.032638,False,2213.411765,0.017345,False,False
10,202303,감자,2296.409091,35.333323,0.015386,False,2251.802679,0.019809,False,False
15,202304,감자,2908.300000,168.443056,0.057918,False,2296.409091,0.266456,False,True
20,202305,감자,3233.650000,433.715469,0.134126,True,2908.300000,0.111869,True,False


In [42]:
fc.plot_cv_mom_analysis(filtered_org, top_items_org, show_plot=True)

Loading BokehJS ...

In [43]:
# 1. 고변동성 기준 (Q3 이상)
q3 = cv_summary_org['cv'].quantile(0.75)
top_cv = cv_summary_org[cv_summary_org['cv'] >= q3]

# 2. 품목 기준 상위 5개만 필터링 (optional)
top_items = sorted(top_cv['pdlt_nm'].value_counts().head(5).index.tolist())
filtered = top_cv[top_cv['pdlt_nm'].isin(top_items)]

# 3. Bokeh plot 설정
p = figure(title="📈 고변동성 품목 월별 단가 추이",
           x_range=sorted(filtered['base_ym'].astype(str).unique().tolist()),
           x_axis_label="기준 월", y_axis_label="평균 단가 (원)",
           width=900, height=450, sizing_mode="stretch_width",toolbar_location="above")

colors = Category10[10]
for i, item in enumerate(top_items):
    df_i = filtered[filtered['pdlt_nm'] == item].sort_values(by='base_ym')
    source = ColumnDataSource(data={
        'x': df_i['base_ym'].astype(str),
        'y': df_i['avg_unit_price'],
        'cv': df_i['cv'],
        '품목': df_i['pdlt_nm']
    })
    p.line('x', 'y', source=source, line_width=2, color=colors[i], legend_label=item)
    p.scatter('x', 'y', source=source, size=6, color=colors[i])

# 4. HoverTool 추가
hover = HoverTool(tooltips=[
    ("품목", "@품목"),
    ("월", "@x"),
    ("평균 단가", "@y{0,0} 원"),
    ("CV", "@cv{0.000}")
])
p.add_tools(hover)
p.legend.click_policy = "hide"

show(p)

#### 2) 도매출하단위 분석 (개)

In [44]:
# 도매출하단위 확인
df_dm['whsl_smt_unit_nm'].unique()

array(['kg', '개', '속', '마리'], dtype=object)

In [45]:
df_1 = df_dm[df_dm['whsl_smt_unit_nm'] == '개'].reset_index(drop=True)
df_1.loc[:, 'unit_price'] = (df_1['pdlt_prce'] / df_1['whsl_smt_unit_mg'])
df_1

,base_ym,prce_reg_ymd,mrkt_esnt_no,mrkt_code,mrkt_nm,ctnp_code,ctnp_nm,sggu_code,sggu_nm,pdlt_code,pdlt_nm,spcs_code,spcs_nm,wsrt_exmn_se_code,exmn_se_nm,bulk_grad_code,bulk_grad_nm,pdlt_prce,whsl_smt_unit_mg,whsl_smt_unit_nm,dcnt_prce_yn,unit_price
0,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,221,수박,0,수박,2,도매,5,중품,15000,1,개,N,15000.0
1,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,221,수박,0,수박,2,도매,4,상품,18000,1,개,N,18000.0
2,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,223,오이,2,다다기계통,2,도매,4,상품,90000,100,개,N,900.0
3,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,223,오이,2,다다기계통,2,도매,5,중품,86000,100,개,N,860.0
4,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,224,호박,1,애호박,2,도매,4,상품,32000,20,개,N,1600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8253,202312,2023-12-29,2,110211,가락도매,11,서울,1101,서울,223,오이,3,취청,2,도매,5,중품,51200,50,개,N,1024.0
8254,202312,2023-12-29,2,110211,가락도매,11,서울,1101,서울,224,호박,1,애호박,2,도매,4,상품,33200,20,개,N,1660.0
8255,202312,2023-12-29,2,110211,가락도매,11,서울,1101,서울,224,호박,1,애호박,2,도매,5,중품,31200,20,개,N,1560.0
8256,202312,2023-12-29,2,110211,가락도매,11,서울,1101,서울,223,오이,2,다다기계통,2,도매,4,상품,147000,100,개,N,1470.0


##### 1) 일자별 평균 도매 단가 계산

In [46]:
price_trend_1 = df_1.groupby(['prce_reg_ymd', 'pdlt_nm'])['unit_price'].mean().reset_index()
price_trend_1

,prce_reg_ymd,pdlt_nm,unit_price
0,2023-01-02,수박,16620.000000
1,2023-01-02,오이,977.214286
2,2023-01-02,호박,1577.000000
3,2023-01-03,수박,17180.000000
4,2023-01-03,오이,941.214286
...,...,...,...
718,2023-12-28,오이,1207.875000
719,2023-12-28,호박,1621.500000
720,2023-12-29,수박,16677.777778
721,2023-12-29,오이,1292.125000


In [47]:
fc.plot_daily_price_with_moving_avg(price_trend_1, window=30)

Loading BokehJS ...

- “시계열 기반 line-plot을 통해 주요 식재료의 일별 도매 가격 추이를 분석하고, 계절성 및 가격 급등락 구간을 식별하였다.”
- 가격 추이 분석 시 계절성 및 시즌별 흐름 확인을 위한 이동평균선(30일) 적용
- 수박은 여름철 수요 증가에 따라 가격 변동성이 크게 나타났으며, 이에 비해 오이와 호박은 상대적으로 안정적인 추세를 유지

##### 2) 월평균 복합 기준 월 단가 추이

In [48]:
cv_summary, top_items, filtered = fc.calculate_cv_summary(price_trend_1)

In [49]:
filtered[(filtered['risk_flag_cv'] == True) | (filtered['risk_flag_mom'] == True)]['base_ym'].unique()

array(['202302', '202303', '202304', '202308', '202309', '202310',
       '202305', '202307', '202311', '202312', '202306'], dtype=object)

In [50]:
filtered.head()

,base_ym,pdlt_nm,avg_unit_price,std_unit_price,cv,high_cv,avg_unit_price_shift,mom_increase,risk_flag_cv,risk_flag_mom
0,202301,수박,17914.117647,687.523262,0.038379,False,NaN,NaN,False,False
3,202302,수박,26275.500000,7189.311127,0.273613,True,17914.117647,0.466748,True,True
6,202303,수박,24663.181818,5463.382335,0.221520,True,26275.500000,-0.061362,True,False
9,202304,수박,19209.000000,1516.321380,0.078938,False,24663.181818,-0.221147,False,True
12,202305,수박,18314.000000,536.415000,0.029290,False,19209.000000,-0.046593,False,False


In [51]:
top_items

['오이', '수박', '호박']

In [52]:
fc.plot_cv_mom_analysis(filtered, top_items, show_plot=True)

Loading BokehJS ...

#### 3) 도매출하단위 분석 (kg)

In [53]:
# 도매출하단위 확인
df_dm['whsl_smt_unit_nm'].unique()

array(['kg', '개', '속', '마리'], dtype=object)

In [54]:
df_2 = df_dm[df_dm['whsl_smt_unit_nm'] == 'kg'].reset_index(drop=True)
df_2.loc[:, 'unit_price'] = (df_2['pdlt_prce'] / df_2['whsl_smt_unit_mg'])
df_2

,base_ym,prce_reg_ymd,mrkt_esnt_no,mrkt_code,mrkt_nm,ctnp_code,ctnp_nm,sggu_code,sggu_nm,pdlt_code,pdlt_nm,spcs_code,spcs_nm,wsrt_exmn_se_code,exmn_se_nm,bulk_grad_code,bulk_grad_nm,pdlt_prce,whsl_smt_unit_mg,whsl_smt_unit_nm,dcnt_prce_yn,unit_price
0,202301,2023-01-02,6,210022,부전,21,부산,2100,부산,241,건고추,2,양건,2,도매,5,중품,750000,30,kg,N,25000.000000
1,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,420,파인애플,2,수입,2,도매,5,중품,26000,12,kg,N,2166.666667
2,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,420,파인애플,2,수입,2,도매,4,상품,29000,12,kg,N,2416.666667
3,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,152,감자,1,수미,2,도매,4,상품,45000,20,kg,N,2250.000000
4,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,418,바나나,2,수입,2,도매,5,중품,24300,13,kg,N,1869.230769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156611,202312,2023-12-29,9,210042,엄궁도매,21,부산,2100,부산,246,파,2,쪽파,2,도매,5,중품,8000,1,kg,N,8000.000000
156612,202312,2023-12-29,9,210042,엄궁도매,21,부산,2100,부산,246,파,2,쪽파,2,도매,4,상품,10000,1,kg,N,10000.000000
156613,202312,2023-12-29,9,210042,엄궁도매,21,부산,2100,부산,246,파,0,대파,2,도매,5,중품,4300,1,kg,N,4300.000000
156614,202312,2023-12-29,7,210031,공동어,21,부산,2100,부산,653,전복,0,전복,2,도매,5,중품,30600,1,kg,N,30600.000000


In [55]:
df_2['spcs_nm'].unique()

array(['양건', '수입', '수미', '월동', '밤', '신고', '오이맛고추', '후지', '단감', '양배추',
       '미나리', '깻잎', '가시계통', '일반계', '냉동(수입)', '풋고추', '꽈리고추', '국산', '중국',
       '백색(국산)', '붉은 팥(국산)', '흰 콩(수입)', '붉은고추', '청양고추', '흰다리(수입)', '전복',
       '토마토', '얼갈이배추', '쪽파', '팽이버섯', '느타리버섯', '난지(대서)', '멜론', '파프리카', '청',
       '대추방울토마토', '대파', '샤인머스켓', '적', '시금치', '새송이버섯', '양파', '딸기',
       '애느타리버섯', '깐마늘(남도)', '깐마늘(대서)', '노지', '흰 콩(국산)', '건미역', '생선', '인도',
       '네이블 미국', '레드글로브 페루', '건멸치', '붉은 팥(수입)', '열무', '세척(수입)', '무세척',
       '쥬키니', '난지(남도)', '냉동', '그린 뉴질랜드', '굴', '메밀(수입)', '화건', '대지마', '시설',
       '참외', '햇양파', '레드글로브 칠레', '봄', '수미(노지)', '20kg', '레드글로브 칠레(~23.5)',
       '양건(~23.5)', '브로콜리(국산)', '알배기배추', '햇깐마늘(대서)', '햇깐마늘(남도)', '거봉',
       '발렌시아 미국', '백도', '여름(고랭지)', '캠벨얼리', '네이블 호주', '고랭지', '쓰가루',
       '쓰가루(아오리)', '원황', '홍로', '햇산화건', 'MBA', '한지', '20kg(햅쌀)',
       '풋고추(녹광 등)', '가을'], dtype=object)

##### 1) 일자별 평균 도매 단가 계산

In [56]:
price_trend_2 = df_2.groupby(['prce_reg_ymd', 'pdlt_nm'])['unit_price'].mean().reset_index()
price_trend_2

,prce_reg_ymd,pdlt_nm,unit_price
0,2023-01-02,갈치,13411.111111
1,2023-01-02,감귤,3028.000000
2,2023-01-02,감자,2258.000000
3,2023-01-02,건고추,21968.333333
4,2023-01-02,건멸치,21400.000000
...,...,...,...
14337,2023-12-29,포도,7355.000000
14338,2023-12-29,풋고추,6731.875000
14339,2023-12-29,피마늘,4585.555556
14340,2023-12-29,피망,8077.000000


In [57]:
fc.plot_daily_price_with_moving_avg(price_trend_2, window=30)

Loading BokehJS ...

##### 2) 월평균 복합 기준 월 단가 추이

In [58]:
cv_summary_2, top_items_2, filtered_2 = fc.calculate_cv_summary(price_trend_2, num_top_items=10)

In [59]:
fc.plot_cv_mom_analysis(filtered_2, top_items_2, show_plot=True)

Loading BokehJS ...

#### 3) 도매출하단위 분석 (속)

In [60]:
# 도매출하단위 확인
df_dm['whsl_smt_unit_nm'].unique()

array(['kg', '개', '속', '마리'], dtype=object)

In [61]:
df_3 = df_dm[df_dm['whsl_smt_unit_nm'] == '속'].reset_index(drop=True)
df_3.loc[:, 'unit_price'] = (df_3['pdlt_prce'] / df_3['whsl_smt_unit_mg'])
df_3

,base_ym,prce_reg_ymd,mrkt_esnt_no,mrkt_code,mrkt_nm,ctnp_code,ctnp_nm,sggu_code,sggu_nm,pdlt_code,pdlt_nm,spcs_code,spcs_nm,wsrt_exmn_se_code,exmn_se_nm,bulk_grad_code,bulk_grad_nm,pdlt_prce,whsl_smt_unit_mg,whsl_smt_unit_nm,dcnt_prce_yn,unit_price
0,202301,2023-01-02,17,240121,양동,24,광주,2401,광주,641,김,0,마른김,2,도매,5,중품,6160,1,속,N,6160.0
1,202301,2023-01-02,25,250116,중부,25,대전,2501,대전,641,김,0,마른김,2,도매,5,중품,5000,1,속,N,5000.0
2,202301,2023-01-02,8,210032,남포동건어물,21,부산,2100,부산,641,김,0,마른김,2,도매,5,중품,6500,1,속,N,6500.0
3,202301,2023-01-02,13,220024,칠성,22,대구,2200,대구,641,김,0,마른김,2,도매,5,중품,6330,1,속,N,6330.0
4,202301,2023-01-02,2,110211,가락도매,11,서울,1101,서울,641,김,0,마른김,2,도매,5,중품,6620,1,속,N,6620.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,202312,2023-12-29,17,240121,양동,24,광주,2401,광주,641,김,0,마른김,2,도매,5,중품,7500,1,속,N,7500.0
1201,202312,2023-12-29,11,220021,북부도매,22,대구,2200,대구,641,김,0,마른김,2,도매,5,중품,7360,1,속,N,7360.0
1202,202312,2023-12-29,25,250116,중부,25,대전,2501,대전,641,김,0,마른김,2,도매,5,중품,7330,1,속,N,7330.0
1203,202312,2023-12-29,2,110211,가락도매,11,서울,1101,서울,641,김,0,마른김,2,도매,5,중품,7000,1,속,N,7000.0


In [62]:
df_3['spcs_nm'].unique()

array(['마른김'], dtype=object)

##### 1) 일자별 평균 도매 단가 계산

In [63]:
price_trend_3 = df_3.groupby(['prce_reg_ymd', 'pdlt_nm'])['unit_price'].mean().reset_index()
fc.plot_daily_price_with_moving_avg(price_trend_3, window=30)

Loading BokehJS ...

##### 2) 월평균 복합 기준 월 단가 추이

In [64]:
cv_summary_3, top_items_3, filtered_3 = fc.calculate_cv_summary(price_trend_3, num_top_items=10)
fc.plot_cv_mom_analysis(filtered_3, top_items_3, show_plot=True)

Loading BokehJS ...

#### 4) 도매출하단위 분석 (마리)

In [65]:
# 도매출하단위 확인
df_dm['whsl_smt_unit_nm'].unique()

array(['kg', '개', '속', '마리'], dtype=object)

In [66]:
df_4 = df_dm[df_dm['whsl_smt_unit_nm'] == '마리'].reset_index(drop=True)
df_4.loc[:, 'unit_price'] = (df_4['pdlt_prce'] / df_4['whsl_smt_unit_mg'])
df_4

,base_ym,prce_reg_ymd,mrkt_esnt_no,mrkt_code,mrkt_nm,ctnp_code,ctnp_nm,sggu_code,sggu_nm,pdlt_code,pdlt_nm,spcs_code,spcs_nm,wsrt_exmn_se_code,exmn_se_nm,bulk_grad_code,bulk_grad_nm,pdlt_prce,whsl_smt_unit_mg,whsl_smt_unit_nm,dcnt_prce_yn,unit_price
0,202301,2023-01-02,17,240121,양동,24,광주,2401,광주,640,건오징어,0,건오징어,2,도매,5,중품,85500,20,마리,N,4275.0
1,202301,2023-01-02,17,240121,양동,24,광주,2401,광주,639,북어,1,황태,2,도매,5,중품,48600,10,마리,N,4860.0
2,202301,2023-01-02,25,250116,중부,25,대전,2501,대전,639,북어,1,황태,2,도매,5,중품,48700,10,마리,N,4870.0
3,202301,2023-01-02,25,250116,중부,25,대전,2501,대전,640,건오징어,0,건오징어,2,도매,5,중품,86200,20,마리,N,4310.0
4,202301,2023-01-02,8,210032,남포동건어물,21,부산,2100,부산,640,건오징어,0,건오징어,2,도매,5,중품,90000,20,마리,N,4500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2405,202312,2023-12-29,25,250116,중부,25,대전,2501,대전,639,북어,1,황태,2,도매,5,중품,49300,10,마리,N,4930.0
2406,202312,2023-12-29,2,110211,가락도매,11,서울,1101,서울,640,건오징어,0,건오징어,2,도매,5,중품,86300,20,마리,N,4315.0
2407,202312,2023-12-29,2,110211,가락도매,11,서울,1101,서울,639,북어,1,황태,2,도매,5,중품,50600,10,마리,N,5060.0
2408,202312,2023-12-29,8,210032,남포동건어물,21,부산,2100,부산,640,건오징어,0,건오징어,2,도매,5,중품,93000,20,마리,N,4650.0


In [67]:
df_4['spcs_nm'].unique()

array(['건오징어', '황태'], dtype=object)

##### 1) 일자별 평균 도매 단가 계산

In [68]:
price_trend_4 = df_4.groupby(['prce_reg_ymd', 'pdlt_nm'])['unit_price'].mean().reset_index()
fc.plot_daily_price_with_moving_avg(price_trend_4, window=30)

Loading BokehJS ...

##### 2) 월평균 복합 기준 월 단가 추이

In [69]:
cv_summary_4, top_items_4, filtered_4 = fc.calculate_cv_summary(price_trend_4, num_top_items=10)
fc.plot_cv_mom_analysis(filtered_4, top_items_4, show_plot=True)

Loading BokehJS ...

In [70]:
cv_summary.shape, filtered.shape

((36, 10), (36, 10))

In [71]:
cv_summary[(cv_summary['risk_flag_cv'] == True) | (cv_summary['risk_flag_mom'] == True)]['pdlt_nm'].unique()

array(['수박', '오이', '호박'], dtype=object)

In [72]:
cv_summary_2[(cv_summary_2['risk_flag_cv'] == True) | (cv_summary_2['risk_flag_mom'] == True)]['pdlt_nm'].unique()

array(['갈치', '감귤', '감자', '고구마', '굴', '깻잎', '느타리버섯', '단감', '당근', '딸기',
       '레몬', '망고', '멜론', '무', '미나리', '방울토마토', '배추', '복숭아', '붉은고추', '브로콜리',
       '사과', '상추', '새송이버섯', '생강', '시금치', '알배기배추', '양배추', '양파', '얼갈이배추',
       '열무', '오이', '참다래', '참외', '체리', '토마토', '파', '파프리카', '팽이버섯', '포도',
       '풋고추', '피마늘', '피망', '호박'], dtype=object)

In [73]:
cv_summary_3[(cv_summary_3['risk_flag_cv'] == True) | (cv_summary_3['risk_flag_mom'] == True)]['pdlt_nm'].unique()

array(['김'], dtype=object)

In [74]:
cv_summary_4[(cv_summary_4['risk_flag_cv'] == True) | (cv_summary_4['risk_flag_mom'] == True)]['pdlt_nm'].unique()

array(['건오징어', '북어'], dtype=object)

### 도매출하단위 통합

In [75]:
final_df = df_dm.reset_index(drop=True).copy()
final_df.loc[:, 'unit_price'] = (final_df['pdlt_prce'] / final_df['whsl_smt_unit_mg'])

In [76]:
final_df

,base_ym,prce_reg_ymd,mrkt_esnt_no,mrkt_code,mrkt_nm,ctnp_code,ctnp_nm,sggu_code,sggu_nm,pdlt_code,pdlt_nm,spcs_code,spcs_nm,wsrt_exmn_se_code,exmn_se_nm,bulk_grad_code,bulk_grad_nm,pdlt_prce,whsl_smt_unit_mg,whsl_smt_unit_nm,dcnt_prce_yn,unit_price
0,202301,2023-01-02,6,210022,부전,21,부산,2100,부산,241,건고추,2,양건,2,도매,5,중품,750000,30,kg,N,25000.000000
1,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,420,파인애플,2,수입,2,도매,5,중품,26000,12,kg,N,2166.666667
2,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,420,파인애플,2,수입,2,도매,4,상품,29000,12,kg,N,2416.666667
3,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,152,감자,1,수미,2,도매,4,상품,45000,20,kg,N,2250.000000
4,202301,2023-01-02,11,220021,북부도매,22,대구,2200,대구,418,바나나,2,수입,2,도매,5,중품,24300,13,kg,N,1869.230769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168484,202312,2023-12-29,9,210042,엄궁도매,21,부산,2100,부산,246,파,2,쪽파,2,도매,5,중품,8000,1,kg,N,8000.000000
168485,202312,2023-12-29,9,210042,엄궁도매,21,부산,2100,부산,246,파,2,쪽파,2,도매,4,상품,10000,1,kg,N,10000.000000
168486,202312,2023-12-29,9,210042,엄궁도매,21,부산,2100,부산,246,파,0,대파,2,도매,5,중품,4300,1,kg,N,4300.000000
168487,202312,2023-12-29,7,210031,공동어,21,부산,2100,부산,653,전복,0,전복,2,도매,5,중품,30600,1,kg,N,30600.000000


#### 1) 일자별 평균 도매 단가 계산

In [77]:
price_trend_final = final_df.groupby(['prce_reg_ymd', 'pdlt_nm'])['unit_price'].mean().reset_index()

In [78]:
fc.plot_daily_price_with_moving_avg(price_trend_final, window=30)

Loading BokehJS ...

In [79]:
cv_summary_final, top_items_final, filtered_final = fc.calculate_cv_summary(price_trend_final, num_top_items=10)

In [80]:
top_items_final

['오이', '피망', '상추', '호박', '붉은고추', '얼갈이배추', '풋고추', '배추', '열무', '파']

In [81]:
price_trend_agg = price_trend_final.copy()
price_trend_agg['base_ym'] = pd.to_datetime(
    price_trend_agg['prce_reg_ymd']).dt.strftime('%Y%m')

# 2. 월별 통계 계산
cv_summary = price_trend_agg.groupby(['base_ym', 'pdlt_nm']).agg(
    avg_unit_price=('unit_price', 'mean'),
    std_unit_price=('unit_price', 'std')
).reset_index()

# 3. 변동계수(CV) 계산
cv_summary['cv'] = cv_summary['std_unit_price'] / \
    cv_summary['avg_unit_price']
cv_summary.sort_values(by=['base_ym', 'pdlt_nm'], inplace=True)

# 4. 고변동성 기준 설정 (Q3)
cv_threshold = cv_summary['cv'].quantile(0.75)
cv_summary['high_cv'] = cv_summary['cv'] >= cv_threshold

# 5. 전월 대비 단가 상승률 계산
cv_summary = cv_summary.sort_values(['pdlt_nm', 'base_ym'])
cv_summary['avg_unit_price_shift'] = cv_summary.groupby(
    'pdlt_nm')['avg_unit_price'].shift(1)
cv_summary['mom_increase'] = (
    cv_summary['avg_unit_price'] - cv_summary['avg_unit_price_shift']
) / cv_summary['avg_unit_price_shift']

# 6. 위험 플래그 설정
cv_summary['risk_flag_cv'] = cv_summary['cv'] >= cv_summary['cv'].quantile(
    0.75)
cv_summary['risk_flag_mom'] = cv_summary['mom_increase'].abs() >= 0.2

# 7. 주요 품목 선정 (고변동성 기준)
high = cv_summary[cv_summary['high_cv']]

In [82]:
low_items = high['pdlt_nm'].value_counts(ascending=True).head(20).index.tolist()

In [83]:
top_items = high['pdlt_nm'].value_counts().index.tolist()

In [84]:
fc.plot_cv_mom_analysis(filtered_final, top_items_final, show_plot=True)

Loading BokehJS ...

In [85]:
filtered_final

,base_ym,pdlt_nm,avg_unit_price,std_unit_price,cv,high_cv,avg_unit_price_shift,mom_increase,risk_flag_cv,risk_flag_mom
31,202301,배추,573.652941,74.706443,0.130229,True,NaN,NaN,True,False
95,202302,배추,633.175000,45.596028,0.072012,False,573.652941,0.103760,False,False
159,202303,배추,811.090909,41.237626,0.050842,False,633.175000,0.280990,False,True
224,202304,배추,995.710000,45.361390,0.045557,False,811.090909,0.227618,False,True
286,202305,배추,907.897143,122.596915,0.135034,True,995.710000,-0.088191,True,False
...,...,...,...,...,...,...,...,...,...,...
516,202308,호박,1567.386364,232.224526,0.148160,True,1374.121250,0.140646,True,False
579,202309,호박,1808.065789,418.834733,0.231648,True,1567.386364,0.153555,True,False
644,202310,호박,1872.197368,321.170835,0.171548,True,1808.065789,0.035470,True,False
710,202311,호박,1525.431818,170.976975,0.112084,True,1872.197368,-0.185218,True,False


In [86]:
len(top_items_final)

10

In [87]:
len(low_items)

20

In [88]:
print(f"변동성이 심한 품목 Top 10:")
print(f"{top_items_final}")
print(f"\n변동성이 낮은 품목 20개:")
print(f"{low_items}")

변동성이 심한 품목 Top 10:
['오이', '피망', '상추', '호박', '붉은고추', '얼갈이배추', '풋고추', '배추', '열무', '파']

변동성이 낮은 품목 20개:
['갈치', '피마늘', '포도', '양파', '새송이버섯', '브로콜리', '배', '딸기', '사과', '단감', '고구마', '굴', '레몬', '망고', '감자', '느타리버섯', '당근', '감귤', '토마토', '체리']


# 📄 월평균 도매 단가 복합 리스크 분석 리포트  
> 기준: **CV 상위 25% 이상** 또는 **MoM ±20% 이상** 변동

---

## 📘 분석 지표 정의

### 📐 CV (Coefficient of Variation)
- **가격의 상대적 변동성**을 나타내는 지표
- 계산식: `CV = 표준편차 / 평균`
- 값이 클수록 **월내 가격의 출렁임이 심함**  
  → 수급 예측이 어려운 품목으로 간주

### 📈 MoM (Month-over-Month 증가율)
- **전월 대비 평균 단가의 상승/하락률**
- 계산식: `(이번달 평균 - 전월 평균) / 전월 평균`
- ±20% 이상은 **급격한 등락**으로 판단하며, 라벨링 처리

---

## ✅ 분석 개요

- **분석 대상**: 수박, 오이, 호박 등 주요 식재료  
- **분석 기간**: 2023년 1월 ~ 12월  
- **분석 기준**:
  - `CV ≥ Q3` (가격 변동계수 상위 25%)
  - `MoM ±20% 이상` 변화 시 **위험 월로 표시**

---

## 📌 주요 인사이트

### 1. 수박
- **2월(+47%)**, **8월(+45%)** 급등 → 계절성 영향 유력
- **9월(-22%)**, **10월(-35%)** 급락 확인
- **계절별 수급 불균형이 크고**, 월간 변동성(CV)도 매우 높음

### 2. 호박
- 3~5월 **연속 하락(-31% ~ -22%)**, 10월(-21%) 급락
- 단가는 낮지만 **CV가 꾸준히 높음** → 예측 불확실성 큼

### 3. 오이
- **7월(+22%)**, **12월(+31%)** 급등
- 평소엔 저가이나, **특정 시기에 급등하는 특성** 주의 필요

---

## 📍 요약 표

| 품목 | 주요 위험 구간 | 특성 요약 |
|------|----------------|------------|
| 수박 | 2월(+47%), 8월(+45%), 10월(-35%) | 강한 계절성 + 큰 등락폭 |
| 호박 | 3~5월(-22%~31%), 10월(-21%) | 낮은 가격이나 지속적 변동 |
| 오이 | 7월(+22%), 12월(+31%) | 저가지만 단기 급등락 존재 |

---

## 🧠 전략적 시사점

- 외식업 창업자는 **식재료 수급 안정성**을 고려한 메뉴 구성 필요
- **도매시장 가격 예측 시스템**과 **대체 식재료 확보 전략**을 병행할 것
- 리스크가 큰 품목은 **계약 재료 구매**, **가공품 대체**, **시기별 프로모션 대응** 등이 필요함

In [89]:
# 📉 MoM / CV 기준 품목별 Top3 랭킹 생성
import pandas as pd

# 전제: cv_summary_1 데이터프레임이 존재한다고 가정
# 1. MoM Top3 랭킹
mom_rank = (
    cv_summary[['base_ym', 'pdlt_nm', 'mom_increase']]
    .dropna()
    .sort_values('mom_increase', ascending=False)
    .groupby('base_ym')
    .head(3)
    .reset_index(drop=True)
)
mom_rank['type'] = 'MoM Top3'

# 2. CV Top3 랭킹
cv_rank = (
    cv_summary[['base_ym', 'pdlt_nm', 'cv']]
    .dropna()
    .sort_values('cv', ascending=False)
    .groupby('base_ym')
    .head(3)
    .reset_index(drop=True)
)
cv_rank['type'] = 'CV Top3'

# 3. 결합 테이블
combined_rank = pd.concat([
    mom_rank.rename(columns={'mom_increase': 'value'}),
    cv_rank.rename(columns={'cv': 'value'})
], ignore_index=True)

# 정렬
combined_rank = combined_rank.sort_values(['base_ym', 'type', 'value'], ascending=[True, True, False]).reset_index(drop=True)
combined_rank

,base_ym,pdlt_nm,value,type
0,202301,상추,0.229103,CV Top3
1,202301,얼갈이배추,0.178052,CV Top3
2,202301,파프리카,0.159872,CV Top3
3,202302,감귤,0.307844,CV Top3
4,202302,붉은고추,0.296440,CV Top3
...,...,...,...,...
64,202312,피망,0.344647,CV Top3
65,202312,파프리카,0.283790,CV Top3
66,202312,딸기,1.315952,MoM Top3
67,202312,체리,0.631570,MoM Top3
